In [18]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import folium 
from datetime import datetime
from math import sin, cos, sqrt, atan2, radians
import sqlite3
import requests
import geopandas as gpd
from utils import get_osrm_distance 
from utils import haversine_distance
import requests
from pyproj import Transformer
import os
from tqdm import tqdm, trange
import bokeh
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, MultiLine
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import export_png
from bokeh.models import LabelSet
from selenium import webdriver


In [19]:
data = pd.read_csv('ChargingStationCleaned.csv')
data['year'] = pd.to_datetime(data['commissioning_date']).dt.year

In [20]:
data.head()

,operator,address,house_number,placeholder1,postcode,city,federal_state,metropolitan_area,latitude_[dg],longitude_[dg],...,number_of_charging_points,type_of_plug_1,p1_[kw],type_of_plug_2,p2_[kw],type_of_plug_3,p3_[kw],type_of_plug_4,p4_[kw],year
0,EnBW mobility+ AG und Co.KG,Hauptstraï¿½e,91c,NaN,73340,Amstetten,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.578534,9.874840,...,2,AC Kupplung Typ 2,43.0,"DC Kupplung Combo, DC CHAdeMO",50.0,0,0.0,0,0.0,2019
1,SWU Energie GmbH,Lindenstraï¿½e,7,NaN,89176,Asselfingen,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.528983,10.193397,...,1,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,0,0.0,2019
2,SWU Energie GmbH,ï¿½llengasse,10,NaN,89177,Ballendorf,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.553539,10.082678,...,1,AC Steckdose Typ 2,22.0,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,2019
3,SWU Energie GmbH,Sterngasse,6,NaN,88481,Balzheim,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.179959,10.076977,...,1,AC Steckdose Typ 2,22.0,0,0.0,0,0.0,0,0.0,2022
4,EnBW mobility+ AG und Co.KG,Heuweg,12,NaN,89179,Beimerstetten,Baden-Wï¿½rttemberg,Landkreis Alb-Donau-Kreis,48.480387,9.980588,...,2,"AC Steckdose Typ 2, AC Schuko",22.0,"AC Steckdose Typ 2, AC Schuko",22.0,0,0.0,0,0.0,2019


In [21]:
def create_network(year, max_distance=100):
    data_year = data[data['year'] <= year]
    n = len(data_year)
    positions = data_year[['latitude_[dg]', 'longitude_[dg]']].to_numpy()

    # Converti le coordinate geografiche in coordinate UTM
    transformer = Transformer.from_crs('epsg:4326', 'epsg:32632', always_xy=True)
    positions_utm = np.array([transformer.transform(x[1], x[0]) for x in positions])

    network = nx.Graph()
    
    # Aggiungi i nodi al grafo con attributi di latitudine e longitudine
    for i in tqdm(range(n), desc="Adding nodes"):
        network.add_node(i + 1, latitude=positions[i, 0], longitude=positions[i, 1])

    for i in tqdm(range(n), desc="Adding edges"):
        for j in range(i + 1, n):  # Evita di ripetere gli archi
            # distance = get_osrm_distance(positions_utm[i, 0], positions_utm[i, 1], positions_utm[j, 0], positions_utm[j, 1])
            distance = get_osrm_distance(positions[i, 0], positions[i, 1], positions[j, 0], positions[j, 1])
            if distance is not None and distance < max_distance * 1000:  # Converti max_distance in metri
                network.add_edge(i + 1, j + 1, weight=distance)

    pos = {i + 1: positions_utm[i] for i in range(n)}

    options = {
        'node_color': 'lavender',
        'node_size': 100,
        'width': 1,
        # 'arrowstyle': '->',
        'arrowsize': 1,
    }

    # # Leggi il file GeoJSON dei confini della Germania
    # germany_boundary = gpd.read_file('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/4090d4e1f89c1184b436b3d9ccaf332b4c5b43d2/2_bundeslaender/4_niedrig.geo.json')

    # # Converti le coordinate geografiche in coordinate UTM
    # germany_boundary = germany_boundary.to_crs('epsg:32632')

    # # Crea un nuovo plot
    # fig, ax = plt.subplots(figsize=(12, 8))

    # # Disegna i confini della Germania
    # germany_boundary.boundary.plot(ax=ax, linewidth=1, color='red')

    # # Disegna il network
    # nx.draw_networkx(network, pos=pos, ax=ax, **options)

    # # Crea le cartelle se non esistono
    # if not os.path.exists("network_graphs_figure"):
    #     os.makedirs("network_graphs_figure")
    # if not os.path.exists("network_graphs_save"):
    #     os.makedirs("network_graphs_save")

    # # Imposta i titoli e visualizza il grafico
    # plt.title(f'EV charging stations network ({year}) of BEV with 100 km range', fontsize=15)
    # plt.show()

    # nx.write_graphml(network, f"network_graphs_save/network_{year}_{max_distance}.graphml")
    
    # # Salva il grafico come file PNG nella cartella "network_graphs_figure"
    # fig.savefig(f"network_graphs_figure/network_{year}_{max_distance}.png", dpi=300, bbox_inches="tight")

    # Leggi il file GeoJSON dei confini della Germania
    germany_boundary = gpd.read_file('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/4090d4e1f89c1184b436b3d9ccaf332b4c5b43d2/2_bundeslaender/4_niedrig.geo.json')

    # Converti le coordinate geografiche in coordinate UTM
    germany_boundary = germany_boundary.to_crs('epsg:32632')



    # ================================================== Graph
    # # Crea un nuovo plot
    # p = figure(x_axis_label='X', y_axis_label='Y', toolbar_location=None)
    # p.xaxis.visible = False
    # p.yaxis.visible = False

    # # Aggiungi gli archi al plot
    # edge_xs = [[pos[edge[0]][0], pos[edge[1]][0]] for edge in network.edges()]
    # edge_ys = [[pos[edge[0]][1], pos[edge[1]][1]] for edge in network.edges()]

    # source_edges = ColumnDataSource(dict(xs=edge_xs, ys=edge_ys))
    # p.add_glyph(source_edges, MultiLine(xs='xs', ys='ys', line_color='black'))

    # # Aggiungi i nodi al plot
    # source_nodes = ColumnDataSource(dict(x=[p[0] for p in pos.values()], y=[p[1] for p in pos.values()]))
    # p.circle(x='x', y='y', size=5, color='lavender', source=source_nodes)

    # # Disegna i confini della Germania
    # for _, row in germany_boundary.iterrows():
    #     if row['geometry'].geom_type == 'Polygon':
    #         xs, ys = row['geometry'].exterior.xy
    #         p.line(xs, ys, color='red', line_width=2)
    #     elif row['geometry'].geom_type == 'MultiPolygon':
    #         for polygon in row['geometry'].geoms:
    #             xs, ys = polygon.exterior.xy
    #             p.line(xs, ys, color='red', line_width=2)

    # # Aggiungi gli archi al plot
    # edge_xs = [[pos[edge[0]][0], pos[edge[1]][0]] for edge in network.edges()]
    # edge_ys = [[pos[edge[0]][1], pos[edge[1]][1]] for edge in network.edges()]

    # source_edges = ColumnDataSource(dict(xs=edge_xs, ys=edge_ys))
    # p.add_glyph(source_edges, MultiLine(xs='xs', ys='ys', line_color='black'))

    # # Aggiungi i nodi al plot
    # source_nodes = ColumnDataSource(dict(x=[p[0] for p in pos.values()], y=[p[1] for p in pos.values()]))
    # p.circle(x='x', y='y', size=5, color='lavender', source=source_nodes)

    # # Aggiungi le etichette dei nodi
    # source_labels = ColumnDataSource(data=dict(x=list(x for x, _ in pos.values()), y=list(y for _, y in pos.values()), text=list(network.nodes())))
    # labels = LabelSet(x='x', y='y', text='text', source=source_labels, text_color='black', text_font_size='8pt', x_offset=5, y_offset=5)
    # p.add_layout(labels)

    # # Visualizza il grafico
    # output_file(f"network_graphs_figure/network_{year}_{max_distance}.html")  # Salva il grafico come file HTML
    # # show(p)  # Mostra il grafico in una nuova finestra del browser

    # # Crea un'istanza del WebDriver di Safari
    # safari_driver = webdriver.Safari()

    # # Salva il grafico come file PNG nella cartella "network_graphs_figure"
    # export_png(p, filename=f"network_graphs_figure/network_bokeh_{year}_{max_distance}.png", webdriver=safari_driver)



In [22]:
# create_network(2009)

In [23]:
# create_network(2010)

In [24]:
# create_network(2011)

In [25]:
# create_network(2012)

In [26]:
# create_network(2013)

In [27]:
# create_network(2014)

In [28]:
# create_network(2015)

In [29]:
# create_network(2016)

In [30]:
# create_network(2017)

In [31]:
# create_network(2018)

In [32]:
# create_network(2019)

In [33]:
# create_network(2020)

Adding edges: 100%|██████████| 18279/18279 [07:55<00:00, 38.42it/s]  


In [34]:
# create_network(2021)

Adding edges:  50%|████▉     | 12533/25252 [7:14:28<7:20:55,  2.08s/it]  


KeyboardInterrupt: 

In [ ]:
create_network(2022)

Adding edges:  50%|████▉     | 14728/29733 [50:15<51:12,  4.88it/s]   


KeyboardInterrupt: 

In [ ]:
lat1, lon1 = 41.9028, 12.4964  # Roma
lat2, lon2 = 45.4642, 9.1900  # Milano

print(haversine_distance(lat1, lon1, lat2, lon2))


476.884684356165


In [ ]:
print(get_osrm_distance(lat1, lon1, lat2, lon2))

476.884684356165


In [ ]:
def main():
    # Coordinate di Monaco di Baviera, Germania
    munich_lat = 48.1351
    munich_lon = 11.5820

    # Coordinate di Berlino, Germania
    berlin_lat = 52.5200
    berlin_lon = 13.4050

    # Calcola la distanza tra Monaco e Berlino utilizzando la funzione get_osrm_distance()
    distance = get_osrm_distance(munich_lat, munich_lon, berlin_lat, berlin_lon)
    print(f"Distanza tra Monaco e Berlino: {distance} km")

if __name__ == "__main__":
    main()


Distanza tra Monaco e Berlino: 504.4153314003782 km
